In [1]:
import os
import pickle
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from youtube_dl import YoutubeDL



In [2]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file =\
r"C:\Users\nguyenducanh\Desktop\youtube_crawler\creds\androdamous.json"
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [3]:
youtube = None
def get_authenticated_service():
    global client_secrets_file
    if os.path.exists("CREDENTIALS_PICKLE_FILE"):
        with open("CREDENTIALS_PICKLE_FILE", 'rb') as f:
            credentials = pickle.load(f)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
        credentials = flow.run_console()
        with open("CREDENTIALS_PICKLE_FILE", 'wb') as f:
            pickle.dump(credentials, f)
    return googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

def search(youtube,name,page_token = None):
    request = youtube.search().list(
            part="id,snippet",
            q=name,pageToken = page_token
        )
    response = request.execute()
    return response



def search_by_pages(name,ite = 1):
    global youtube
    youtube = get_authenticated_service()
    page_token = None
    re = []
    for i in range(ite):
        res = search(youtube,name,page_token)
        page_token = dict(res.items())['nextPageToken']
        re.append(res)
    return re

def fetch(youtube,videoId, page_token = None):
    request = youtube.commentThreads().list(part='id,snippet',
                                            videoId = videoId,
                                           pageToken = page_token)
    response = request.execute()
    return response

def get_comment_by_pages(name, ite = 1):
    global youtube
    x = search_by_pages(name,1)
    videoId = x[0]['items'][0]['id']['videoId']
    page_token = None
    re = []
    for i in range(ite):
        res = fetch(youtube,videoId,page_token)
        page_token = res['nextPageToken']
        re.append(res)
    return re

In [4]:
def download_youtube_audio(path,name,ite):
    res = search_by_pages(name,ite)
    items = []
    for page in res:
        tmp = []
        for ele in page['items']:
            if ele['id'].__contains__('videoId'):
                    tmp.append(ele['id']['videoId'])
        items.extend(tmp)
    urls = [os.path.join('https://youtu.be/',idx) for idx in items]
    d = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    'outtmpl':os.path.join(path,'%(title)s-%(id)s.%(ext)s')
    }
    downloader = YoutubeDL(d)
    for url in urls:
        downloader.extract_info(url)

def fetch_by_genre(path,genre_name,ite):
    folder_path = os.path.join(path, genre_name)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    download_youtube_audio(folder_path, genre_name,ite)
        

In [5]:

s = 'C:\data'
if not os.path.exists(s):
    os.mkdir(s)

In [9]:
fetch_by_genre(s,"Ca trù Việt Nam",10)

[youtube] y-05_1UUu1Y: Downloading webpage
[download] Destination: C:\data\Ca trù Việt Nam\Những Làn Điệu Dân Ca - Hát Văn, Hát Xẩm, Hát Ca Trù Việt Nam Hay Nhất 217-y-05_1UUu1Y.webm
[download] 100% of 61.56MiB in 00:07                          
[ffmpeg] Destination: C:\data\Ca trù Việt Nam\Những Làn Điệu Dân Ca - Hát Văn, Hát Xẩm, Hát Ca Trù Việt Nam Hay Nhất 217-y-05_1UUu1Y.wav
Deleting original file C:\data\Ca trù Việt Nam\Những Làn Điệu Dân Ca - Hát Văn, Hát Xẩm, Hát Ca Trù Việt Nam Hay Nhất 217-y-05_1UUu1Y.webm (pass -k to keep)
[youtube] YzlOUwuMx8k: Downloading webpage
[download] Destination: C:\data\Ca trù Việt Nam\Những Bài Hát Ca Trù Hay Nhất Tuyển chọn-YzlOUwuMx8k.webm
[download] 100% of 111.10MiB in 00:13                          
[ffmpeg] Destination: C:\data\Ca trù Việt Nam\Những Bài Hát Ca Trù Hay Nhất Tuyển chọn-YzlOUwuMx8k.wav
Deleting original file C:\data\Ca trù Việt Nam\Những Bài Hát Ca Trù Hay Nhất Tuyển chọn-YzlOUwuMx8k.webm (pass -k to keep)
[youtube] 41z5HiWIAOs

[download] 100% of 99.86MiB in 00:14                          
[ffmpeg] Destination: C:\data\Ca trù Việt Nam\LK Nhạc Trữ Tình Bolero - Những Ca Khúc Nhạc Vàng Trữ Tình Hay Nhất-zBtOrxf61M0.wav
Deleting original file C:\data\Ca trù Việt Nam\LK Nhạc Trữ Tình Bolero - Những Ca Khúc Nhạc Vàng Trữ Tình Hay Nhất-zBtOrxf61M0.webm (pass -k to keep)
[youtube] Ae8p5r-Ux8M: Downloading webpage
[download] Destination: C:\data\Ca trù Việt Nam\CA TRU ( Xam ) - Viet Nam Music nature song by a teen-Ae8p5r-Ux8M.m4a
[download] 100% of 5.49MiB in 00:00                  
[ffmpeg] Correcting container in "C:\data\Ca trù Việt Nam\CA TRU ( Xam ) - Viet Nam Music nature song by a teen-Ae8p5r-Ux8M.m4a"
[ffmpeg] Destination: C:\data\Ca trù Việt Nam\CA TRU ( Xam ) - Viet Nam Music nature song by a teen-Ae8p5r-Ux8M.wav
Deleting original file C:\data\Ca trù Việt Nam\CA TRU ( Xam ) - Viet Nam Music nature song by a teen-Ae8p5r-Ux8M.m4a (pass -k to keep)
[youtube] VtCeFMuEFDM: Downloading webpage
[youtube] VtCeFMuEF

[youtube] nYdp7EwGL2s: Downloading API JSON


ERROR: Unable to download API page: <urlopen error [Errno 11001] getaddrinfo failed> (caused by URLError(gaierror(11001, 'getaddrinfo failed')))


DownloadError: ERROR: Unable to download API page: <urlopen error [Errno 11001] getaddrinfo failed> (caused by URLError(gaierror(11001, 'getaddrinfo failed')))

In [13]:
x = """These 3 barcharts compare the levels of participation in education and science of developing countries and the developed ones in 1980 and 1990

Principally, the figures for industrialized countries are much higher than those of the developing ones in this period.

As we can see, in 1980 the average years of schooling for industrialised countries was 8 as compared to only 2 for the developing ones. This figure for industrialised countries increased by 2 from 1980 to 1990 while developing countries’ figure has a insignificant increase by about only a year in this period.

Similarly,  from 1980 to 1990 the rate of science workers per 1000 people of industrialised countries increased sharply from 40 to 70 while this number for developing countries was still lower than the level of 20 people in both 1980 and 1990

as regard to the RnD’fund, this number of developing countries halved from 50 to about 25 while developed ones’ figure double from 170 to 340 from 1980 to 1990 
"""
len(x.split())

165